## 生成对应Json表情包文件

### Artalk

In [1]:
import os
import json

def generate_artalk_emojis(base_url, emoji_folders, output_path='.json/artalk-emoji.json', origin=None):
    """
    生成 Artalk 表情包 JSON 文件
    
    参数:
        base_url: 基础 URL，例如 'https://fastly.jsdelivr.net/gh/willow-god/owo'
        emoji_folders: 包含表情包的文件夹列表
        output_path: 输出 JSON 文件的路径
        origin: 原始表情包数据，包含网址到名称的映射关系
    """
    result = []
    
    # 创建网址到名称的映射字典
    url_to_key_map = {}
    if origin:
        for group in origin:
            for item in group.get("items", []):
                if "val" in item and "key" in item:
                    # 去除可能存在的反引号
                    val = item["val"].replace("`", "").strip()
                    url_to_key_map[val] = item["key"]
    
    for folder in emoji_folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个文件夹，已跳过")
            continue
        
        emoji_group = {
            "name": folder,
            "type": "image",
            "items": []
        }
        
        # 获取文件夹中的所有文件
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        
        for file in files:
            # 获取文件名（不带扩展名）
            filename_without_ext = os.path.splitext(file)[0]
            
            # 构建完整URL
            full_url = f"{base_url}/{folder}/{file}"
            
            # 检查是否在原始数据中存在该URL，如果存在则使用原始key
            key = url_to_key_map.get(full_url, filename_without_ext)
            
            # 创建表情项
            emoji_item = {
                "key": key,
                "val": full_url
            }
            
            emoji_group["items"].append(emoji_item)
        
        if emoji_group["items"]:  # 只添加非空的表情组
            result.append(emoji_group)
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 写入 JSON 文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"表情包 JSON 文件已生成: {output_path}")

if __name__ == "__main__":
    base_url = "https://fastly.jsdelivr.net/gh/artemisia1107/owo"
    folders = ["liushen", "blobcat", "linedog", "bilibili","heybox_cube","heybox_heniang"]
    origin_json = ".json/artalk-emoji.json"
    output = ".json/artalk-emoji.json"
    
    # 加载原始表情包数据（如果存在）
    origin_data = None
    try:
        with open(origin_json, 'r', encoding='utf-8') as f:
            origin_data = json.load(f)
        print(f"已加载原始表情包数据: {origin_json}")
    except FileNotFoundError:
        print(f"未找到原始表情包数据文件: {origin_json}")
    except json.JSONDecodeError:
        print(f"原始表情包数据文件格式错误: {origin_json}")
    
    generate_artalk_emojis(base_url, folders, output, origin_data)

已加载原始表情包数据: .json/artalk-emoji.json
警告: liushen 不是一个文件夹，已跳过
表情包 JSON 文件已生成: .json/artalk-emoji.json


### Twikoo

In [2]:
import os
import json

def generate_twikoo_emojis(base_url, emoji_folders, output_path='.json/twikoo-emojis.json', origin=None):
    """
    生成 Twikoo 表情包 JSON 文件
    
    参数:
        base_url: 基础 URL，例如 'https://fastly.jsdelivr.net/gh/willow-god/owo'
        emoji_folders: 包含表情包的文件夹列表
        output_path: 输出 JSON 文件的路径
        origin: 原始表情包数据，包含网址到名称的映射关系
    """
    result = {}
    
    # 创建网址到名称的映射字典
    url_to_text_map = {}
    if origin:
        for category, data in origin.items():
            if "container" in data:
                for item in data["container"]:
                    if "icon" in item and "text" in item:
                        # 从 icon 中提取 URL
                        import re
                        url_match = re.search(r"src=['\"]([^'\"]+)['\"]", item["icon"])
                        if url_match:
                            url = url_match.group(1).replace("`", "").strip()
                            url_to_text_map[url] = item["text"]
    
    for folder in emoji_folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个文件夹，已跳过")
            continue
        
        # 为每个文件夹创建一个类别
        category_name = folder
        # 可以根据文件夹名称自定义类别显示名称
        display_names = {
            "blobcat": "可爱猫",
            "linedog": "线条狗",
            "bilibili": "小电视",
            "liushen": "流沙",
            # 可以添加更多映射
        }
        
        category_display_name = display_names.get(folder, folder)
        
        emoji_category = {
            "type": "image",
            "container": []
        }
        
        # 获取文件夹中的所有文件
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        
        for file in files:
            # 获取文件名（不带扩展名）
            filename_without_ext = os.path.splitext(file)[0]
            
            # 构建完整URL
            full_url = f"{base_url}/{folder}/{file}"
            
            # 检查是否在原始数据中存在该URL，如果存在则使用原始text
            text = url_to_text_map.get(full_url, f"{filename_without_ext}")
            
            # 创建表情项
            emoji_item = {
                "text": text,
                "icon": f"<img src='{full_url}'>"
            }
            
            emoji_category["container"].append(emoji_item)
        
        if emoji_category["container"]:  # 只添加非空的表情组
            result[category_display_name] = emoji_category
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 写入 JSON 文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"Twikoo 表情包 JSON 文件已生成: {output_path}")

if __name__ == "__main__":
    base_url = "https://fastly.jsdelivr.net/gh/artemisia1107/owo"
    folders = ["blobcat", "linedog", "bilibili","heybox_cube","heybox_heniang"]
    origin_json = ".json/twikoo.json"
    output = ".json/twikoo-emoji.json"
    
    # 加载原始表情包数据（如果存在）
    origin_data = None
    try:
        with open(origin_json, 'r', encoding='utf-8') as f:
            origin_data = json.load(f)
        print(f"已加载原始表情包数据: {origin_json}")
    except FileNotFoundError:
        print(f"未找到原始表情包数据文件: {origin_json}")
    except json.JSONDecodeError:
        print(f"原始表情包数据文件格式错误: {origin_json}")
    
    generate_twikoo_emojis(base_url, folders, output, origin_data)

未找到原始表情包数据文件: .json/twikoo.json
Twikoo 表情包 JSON 文件已生成: .json/twikoo-emoji.json


### Waline

In [3]:
from contextlib import nullcontext
import os
import json
import re

def generate_waline_emoji_info(base_url, gh_proxy, emoji_folders, output_dir='.json/waline'):
    """
    为 Waline 评论系统生成表情包配置
    
    参数:
        base_url: 基础 URL，例如 'https://fastly.jsdelivr.net/gh/ljxme/owo'
        emoji_folders: 包含表情包的文件夹列表
        output_dir: 输出目录，为每个表情包创建子目录和 info.json
    """
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 表情包显示名称映射
    display_names = {
        "blobcat": "可爱猫",
        "linedog": "线条狗", 
        "bilibili": "小电视",
        "heybox_cube": "小黑盒方块",
        "heybox_heniang": "小黑盒盒娘"
    }
    
    result = {}

    for folder in emoji_folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个文件夹，已跳过")
            continue
        
        # 创建表情包子目录
        emoji_dir = os.path.join(output_dir, folder)
        os.makedirs(emoji_dir, exist_ok=True)
        
        # 获取文件夹中的所有文件
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        
        # 过滤出图片文件
        image_files = []
        supported_formats = ['.png', '.jpg', '.jpeg', '.gif', '.webp']
        for file in files:
            if any(file.lower().endswith(ext) for ext in supported_formats):
                image_files.append(file)
        
        if not image_files:
            print(f"警告: {folder} 中没有找到图片文件，已跳过")
            continue
        
        # 获取表情名称列表（去掉扩展名，并去除前缀部分）
        emoji_names = []
        for file in image_files:
            name_without_ext = os.path.splitext(file)[0]
            emoji_names.append(name_without_ext)
        # 确定图片格式
        first_file = image_files[0]
        file_ext = os.path.splitext(first_file)[1].lower()
        if file_ext in ['.png', '.jpg', '.jpeg']:
            image_type = file_ext[1:]  # 去掉点号
        elif file_ext == '.gif':
            image_type = 'gif'
        else:
            image_type = 'png'  # 默认格式
        
        # 创建 info.json 文件
        info_data = {
            "name": display_names.get(folder, folder),
            "folder": f"{gh_proxy}/tree/main/{folder}",
            "prefix": f"{folder}-",  # 保留前缀用于 Waline 使用
            "type": image_type,
            # "icon": emoji_names[0] if emoji_names else "default",
            "items": emoji_names
        }
        
        # 构建表情项（按照官方文档要求，包含 text 和 url）
        # for emoji_name in emoji_names:
            # full_url = f"{base_url}/{folder}/{emoji_name}.{image_type}"
            # emoji_item = {
                # "text": emoji_name,  # 仅使用文件名作为表情文本
                # "url": full_url      # 使用完整的 URL
            # }
            # info_data["items"].append(emoji_item)
        
        # 写入 info.json
        info_path = os.path.join(emoji_dir, "info.json")
        with open(info_path, 'w', encoding='utf-8') as f:
            json.dump(info_data, f, ensure_ascii=False, indent=2)
        
        print(f"已生成 Waline 表情包配置: {info_path}")
        print(f"  - 名称: {info_data['name']}")
        print(f"  - 前缀: {info_data['prefix']}")
        print(f"  - 类型: {info_data['type']}")
        print(f"  - 表情数量: {len(info_data['items'])}")
        print()
    
    print(f"Waline 表情包配置生成完成！")
    print(f"配置目录: {output_dir}")
    print(f"使用方法: 在 Waline 初始化时添加 emoji 选项: ")
    print(f"Waline.init(")
    print(f"  el: '#waline',")
    print(f"  serverURL: '您的 Waline 服务端地址',")
    print(f"  emoji: [")
    for folder in emoji_folders:
        if os.path.isdir(folder):
            print(f"'{base_url}/.json/waline/{folder}/',")
    print(f"  ]")
    print(f");")

if __name__ == "__main__":
    base_url = "https://fastly.jsdelivr.net/gh/ljxme/owo"
    gh_proxy = "https://xget.artemisia.icu/gh/ljxme/owo"
    folders = ["blobcat", "linedog", "bilibili", "heybox_cube", "heybox_heniang"]
    
    generate_waline_emoji_info(base_url, gh_proxy, folders)


已生成 Waline 表情包配置: .json/waline\blobcat\info.json
  - 名称: 可爱猫
  - 前缀: blobcat-
  - 类型: png
  - 表情数量: 54

已生成 Waline 表情包配置: .json/waline\linedog\info.json
  - 名称: 线条狗
  - 前缀: linedog-
  - 类型: gif
  - 表情数量: 43

已生成 Waline 表情包配置: .json/waline\bilibili\info.json
  - 名称: 小电视
  - 前缀: bilibili-
  - 类型: png
  - 表情数量: 21

已生成 Waline 表情包配置: .json/waline\heybox_cube\info.json
  - 名称: 小黑盒方块
  - 前缀: heybox_cube-
  - 类型: png
  - 表情数量: 78

已生成 Waline 表情包配置: .json/waline\heybox_heniang\info.json
  - 名称: 小黑盒盒娘
  - 前缀: heybox_heniang-
  - 类型: png
  - 表情数量: 24

Waline 表情包配置生成完成！
配置目录: .json/waline
使用方法: 在 Waline 初始化时添加 emoji 选项: 
Waline.init(
  el: '#waline',
  serverURL: '您的 Waline 服务端地址',
  emoji: [
'https://fastly.jsdelivr.net/gh/ljxme/owo/.json/waline/blobcat/',
'https://fastly.jsdelivr.net/gh/ljxme/owo/.json/waline/linedog/',
'https://fastly.jsdelivr.net/gh/ljxme/owo/.json/waline/bilibili/',
'https://fastly.jsdelivr.net/gh/ljxme/owo/.json/waline/heybox_cube/',
'https://fastly.jsdelivr.net/gh/ljxme/ow

## 图片处理部分

### 压缩指定文件夹列表中的所有图片到指定尺寸

In [4]:
import os
from PIL import Image
import glob

def resize_images(folders, max_size=200, output_suffix="_resized"):
    """
    将指定文件夹中的所有图片进行等比缩小，保持最大边等于指定尺寸，并保留透明区域
    
    参数:
        folders: 包含图片的文件夹列表
        max_size: 缩放后的最大边长（像素）
        output_suffix: 输出文件的后缀，如果为None则覆盖原文件
    
    返回:
        处理成功的图片数量
    """
    supported_formats = ['.png', '.jpg', '.jpeg', '.gif', '.webp']
    processed_count = 0
    
    for folder in folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个有效的文件夹，已跳过")
            continue
            
        print(f"处理文件夹: {folder}")
        
        # 获取文件夹中所有支持格式的图片
        image_files = []
        for ext in supported_formats:
            image_files.extend(glob.glob(os.path.join(folder, f"*{ext}")))
            image_files.extend(glob.glob(os.path.join(folder, f"*{ext.upper()}")))
        
        for img_path in image_files:
            try:
                # 打开图片
                img = Image.open(img_path)
                
                # 获取原始尺寸
                width, height = img.size
                
                # 如果图片已经小于或等于目标尺寸，则跳过
                if width <= max_size and height <= max_size:
                    print(f"跳过 {img_path} (已经小于或等于目标尺寸)")
                    # continue
                
                # 计算缩放比例
                ratio = min(max_size / width, max_size / height)
                new_width = int(width * ratio)
                new_height = int(height * ratio)
                
                # 等比缩放图片
                resized_img = img.resize((max_size, max_size), Image.LANCZOS)
                
                # 确定输出路径
                if output_suffix:
                    filename, ext = os.path.splitext(img_path)
                    output_path = f"{filename}{output_suffix}{ext}"
                else:
                    output_path = img_path
                
                # 保存图片，保留原始格式和透明度
                if img.mode == 'RGBA':
                    # 保留透明通道
                    resized_img.save(output_path, format=img.format, quality=95)
                else:
                    # 普通图片
                    resized_img.save(output_path, format=img.format, quality=95)
                
                processed_count += 1
                print(f"已处理: {img_path} -> {output_path} ({width}x{height} -> {new_width}x{new_height})")
                
            except Exception as e:
                print(f"处理图片失败 {img_path}: {e}")
    
    print(f"处理完成! 共处理 {processed_count} 张图片")
    return processed_count

# 示例用法
if __name__ == "__main__":
    # 要处理的文件夹列表
    folders_to_process = ["liushen", "bilibili"] # 其他均为动态图像，不太好处理
    
    # 设置最大尺寸为150像素
    max_size = 150
    
    # 处理图片，生成带有_resized后缀的新文件
    resize_images(folders_to_process, max_size, None)
    
    # 如果想要覆盖原文件，可以将output_suffix设为None
    # resize_images(folders_to_process, max_size, None)

警告: liushen 不是一个有效的文件夹，已跳过
处理文件夹: bilibili
跳过 bilibili\doge.png (已经小于或等于目标尺寸)
已处理: bilibili\doge.png -> bilibili\doge.png (150x150 -> 150x150)
跳过 bilibili\亲亲.png (已经小于或等于目标尺寸)
已处理: bilibili\亲亲.png -> bilibili\亲亲.png (150x150 -> 150x150)
跳过 bilibili\偷笑.png (已经小于或等于目标尺寸)
已处理: bilibili\偷笑.png -> bilibili\偷笑.png (150x150 -> 150x150)
跳过 bilibili\再见.png (已经小于或等于目标尺寸)
已处理: bilibili\再见.png -> bilibili\再见.png (150x150 -> 150x150)
跳过 bilibili\冷漠.png (已经小于或等于目标尺寸)
已处理: bilibili\冷漠.png -> bilibili\冷漠.png (150x150 -> 150x150)
跳过 bilibili\发怒.png (已经小于或等于目标尺寸)
已处理: bilibili\发怒.png -> bilibili\发怒.png (150x150 -> 150x150)
跳过 bilibili\发财.png (已经小于或等于目标尺寸)
已处理: bilibili\发财.png -> bilibili\发财.png (150x150 -> 150x150)
跳过 bilibili\可爱.png (已经小于或等于目标尺寸)
已处理: bilibili\可爱.png -> bilibili\可爱.png (150x150 -> 150x150)
跳过 bilibili\吐血.png (已经小于或等于目标尺寸)
已处理: bilibili\吐血.png -> bilibili\吐血.png (150x150 -> 150x150)
跳过 bilibili\呆.png (已经小于或等于目标尺寸)
已处理: bilibili\呆.png -> bilibili\呆.png (150x150 -> 150x150)
跳过 bilibili\呕吐